In [450]:
# Para no tener que reiniciar el kernel
%load_ext autoreload
%autoreload 2
import sys  # System-specific parameters and functions
sys.path.append("../")  # Adding the parent directory to the system path
from tqdm import tqdm
from src import sp_nlp as sn
import pandas as pd
# Para trabajar con Mongo
# ------------------------------------------------------------------------------
import pymongo


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from bs4 import BeautifulSoup
import requests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [451]:
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
peliculas = imdb['Peliculas']
actores = imdb['Actores']
api = imdb['Api']
nlp = imdb['NLP']

In [452]:
df =pd.DataFrame(nlp.find())
# Convertir listas de géneros en una cadena de texto
df["genre_text"] = df["genre"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
df["director_text"] = df["direccion"].apply(lambda x: ",".join(x) if isinstance(x, list) else x).apply(lambda x: x.replace(' ','') if type(x) == str else x).apply(lambda x: x.replace(',',' ') if type(x) == str else x)
df["guion_text"] = df["guion"].apply(lambda x: ",".join(x) if isinstance(x, list) else x).apply(lambda x: x.replace(' ','') if type(x) == str else x).apply(lambda x: x.replace(',',' ') if type(x) == str else x)
df["actor_text"] = df["actores"].apply(lambda x: ",".join(x) if isinstance(x, list) else x).apply(lambda x: x.replace(' ','') if type(x) == str else x).apply(lambda x: x.replace(',',' ') if type(x) == str else x)

In [453]:
df.dropna(inplace=True)

In [454]:
# Crear el vectorizador TF-IDF
vectorizer =  TfidfVectorizer(stop_words="english")
tfidf_matrix_arg = vectorizer.fit_transform(df["argumento"])

# Calcular la matriz de similitud de coseno
sim_arg = cosine_similarity(tfidf_matrix_arg, tfidf_matrix_arg)


In [455]:
tfidf_matrix_gen = vectorizer.fit_transform(df['genre_text'])

# Calcular la matriz de similitud de coseno
sim_gen = cosine_similarity(tfidf_matrix_gen, tfidf_matrix_gen)


In [456]:
tfidf_matrix_dir = vectorizer.fit_transform(df['director_text'])

# Calcular la matriz de similitud de coseno
sim_dir = cosine_similarity(tfidf_matrix_dir, tfidf_matrix_dir)


In [457]:
tfidf_matrix_gui = vectorizer.fit_transform(df['guion_text'])

# Calcular la matriz de similitud de coseno
sim_gui = cosine_similarity(tfidf_matrix_gui, tfidf_matrix_gui)

In [458]:
tfidf_matrix_act = vectorizer.fit_transform(df['actor_text'])

# Calcular la matriz de similitud de coseno
sim_act = cosine_similarity(tfidf_matrix_act, tfidf_matrix_act)

In [459]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df["puntuacion"] = scaler.fit_transform(df[["puntuacion"]])


In [460]:
sim_pun = cosine_similarity(df[["puntuacion"]])

In [490]:
df.head(1)

,_id,id_IMDB,argumento,genre,direccion,guion,actores,puntuacion,genre_text,director_text,guion_text,actor_text
0,679e542897e40febc41dc85e,tt0102921,many people run hong kong early day prc two yo...,[Action],[Siu-Hung Chung],"[Gwing-Gai Lee, Ying-Chiet Lee]","[Ray Lui, Kent Cheng, Nina Li Chi, Michael Wai...",0.623529,Action,Siu-HungChung,Gwing-GaiLee Ying-ChietLee,RayLui KentCheng NinaLiChi MichaelWai-ManChan ...


In [504]:
def recomendar_peliculas(titulo, df, cosine_sim):
    # Obtener el índice de la película en el DataFrame
    id_IMDB = peliculas.find_one({'id_IMDB':api.find_one({'title': titulo})['id']})['id_IMDB']
    idx = df[df["id_IMDB"] == id_IMDB].index[0]
    # Obtener las puntuaciones de similitud con todas las películas
    sim_scores = list(enumerate(cosine_sim[idx]))
    print(sim_scores)
    
    # Ordenar las películas por similitud (de mayor a menor)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    print(sim_scores)
    
    # Obtener los índices de las 5 películas más similares (excluyendo la original)
    top_peliculas = sim_scores[0:6]

    print(top_peliculas)

    # Retornar los títulos de las películas recomendadas
    return df.iloc[[i[0] for i in top_peliculas]]["id_IMDB"].values


In [522]:
import numpy as np

In [524]:
df.iloc[[i[0] for i in[(579, np.float64(2.0)), (1639, np.float64(1.2118087097030652)), (1787, np.float64(1.139968611627946)), (1612, np.float64(1.1048694576086289)), (693, np.float64(1.0850119470889759)), (2599, np.float64(1.0844655273725983))]]]["id_IMDB"].values

array(['tt0419706', 'tt0278316', 'tt0796212', 'tt0383203', 'tt0977658',
       'tt2385169'], dtype=object)

In [517]:
cosine_sim = sum([sim_arg, sim_gen])

In [518]:
id_pelis = recomendar_peliculas("Batman Begins", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[(0, np.float64(1.0118278891820767)), (1, np.float64(1.0366412652255637)), (2, np.float64(1.0)), (3, np.float64(1.0)), (4, np.float64(1.0)), (5, np.float64(1.0)), (6, np.float64(1.0)), (7, np.float64(0.0)), (8, np.float64(1.030150627606111)), (9, np.float64(0.0)), (10, np.float64(1.0)), (11, np.float64(1.0423785015539027)), (12, np.float64(1.0)), (13, np.float64(0.0)), (14, np.float64(1.0)), (15, np.float64(1.0)), (16, np.float64(1.0)), (17, np.float64(1.0)), (18, np.float64(0.0)), (19, np.float64(0.0)), (20, np.float64(1.0)), (21, np.float64(0.4854855799853253)), (22, np.float64(0.4169553377465492)), (23, np.float64(1.0196211069137155)), (24, np.float64(1.0)), (25, np.float64(0.026590094662280805)), (26, np.float64(1.019390605916204)), (27, np.float64(1.0)), (28, np.float64(1.0)), (29, np.float64(1.0)), (30, np.float64(1.0)), (31, np.float64(1.0)), (32, np.float64(1.0)), (33, np.float64(0.0)), (34, np.float64(1.0)), (35, np.float64(1.0)), (36, np.float64(1.0)), (37, np.float64(1.0)), 

[{'title': 'Doom'},
 {'title': 'Interception'},
 {'title': 'Bangis'},
 {'title': 'Armorines: Project S.W.A.R.M.'},
 {'title': 'Fong juk'},
 {'title': 'Perfect: Android Rising'}]

In [516]:
id_pelis = recomendar_peliculas("Tarzan", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[(0, np.float64(0.0)), (1, np.float64(0.0)), (2, np.float64(0.0)), (3, np.float64(0.0)), (4, np.float64(0.0)), (5, np.float64(0.03956706248451934)), (6, np.float64(0.0)), (7, np.float64(0.018497281315004804)), (8, np.float64(0.0)), (9, np.float64(0.0)), (10, np.float64(0.0)), (11, np.float64(0.0)), (12, np.float64(0.030867213935768092)), (13, np.float64(0.0)), (14, np.float64(0.0)), (15, np.float64(0.0840484431566764)), (16, np.float64(0.0)), (17, np.float64(0.03673455585249567)), (18, np.float64(0.0)), (19, np.float64(0.0)), (20, np.float64(0.0)), (21, np.float64(0.0)), (22, np.float64(0.0)), (23, np.float64(0.0)), (24, np.float64(0.0)), (25, np.float64(0.031498468555036706)), (26, np.float64(0.0)), (27, np.float64(0.0)), (28, np.float64(0.028746297363453747)), (29, np.float64(0.03141965549407512)), (30, np.float64(0.0)), (31, np.float64(0.0)), (32, np.float64(0.0)), (33, np.float64(0.0)), (34, np.float64(0.02512435868859281)), (35, np.float64(0.0)), (36, np.float64(0.0)), (37, np.flo

[{'title': 'Freak Out'},
 {'title': 'Hatred of a Minute'},
 {'title': 'Sarinja-ng-Nangam'},
 {'title': 'Kalaignan'},
 {'title': 'Cornered'},
 {'title': 'BlackMale'}]

In [515]:
id_pelis = recomendar_peliculas("Avengers", df, cosine_sim)
list(api.find({'id':{'$in':list(id_pelis)}},{"title": 1,"_id": 0}))

[(0, np.float64(0.0)), (1, np.float64(0.0)), (2, np.float64(0.0)), (3, np.float64(0.0)), (4, np.float64(0.0)), (5, np.float64(0.061674088949866736)), (6, np.float64(0.10852226846138795)), (7, np.float64(0.0)), (8, np.float64(0.0)), (9, np.float64(0.0)), (10, np.float64(0.0)), (11, np.float64(0.0)), (12, np.float64(0.0)), (13, np.float64(0.0)), (14, np.float64(0.0)), (15, np.float64(0.046471266796100176)), (16, np.float64(0.0)), (17, np.float64(0.0)), (18, np.float64(0.0)), (19, np.float64(0.0345712652736525)), (20, np.float64(0.04152074851047608)), (21, np.float64(0.04990306665580905)), (22, np.float64(0.026293018436865857)), (23, np.float64(0.0)), (24, np.float64(0.0)), (25, np.float64(0.0)), (26, np.float64(0.0)), (27, np.float64(0.0)), (28, np.float64(0.0)), (29, np.float64(0.0)), (30, np.float64(0.0)), (31, np.float64(0.0)), (32, np.float64(0.0)), (33, np.float64(0.03146146144456594)), (34, np.float64(0.0)), (35, np.float64(0.0)), (36, np.float64(0.0)), (37, np.float64(0.0)), (38, 

[{'title': 'Monolith'},
 {'title': 'Vaastav: The Reality'},
 {'title': 'Numbered'},
 {'title': 'Housefull 2'},
 {'title': 'Diamond Ruff'},
 {'title': 'Fly Light'}]

In [320]:
import random

In [334]:
random.choice(list(api.find()))['id']

'tt10055650'

In [336]:
random.choice(list(peliculas.find()))['id_IMDB']

'tt11684950'

In [514]:
api.find_one({'id':random.choice(list(peliculas.find()))['id_IMDB']})

{'_id': ObjectId('679b98700c8ad3cc3b346ffc'),
 'id': 'tt0681118',
 'title': 'Future Boy - October 6, 1957',
 'type': 'TV Episode',
 'genre': ['Adventure'],
 'year': 1991}